# Ejemplo 1, Nuevo proyecto

1. crear nuevo proyecto

In [ ]:
$scrapy startproject tutorial

2. en la carpeta tutorial/spiders crear un fichero con nombre quotes_spider.py con el siguiente contenido:

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            'http://quotes.toscrape.com/page/1/',
            'http://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = 'quotes-%s.html' % page
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log('Saved file %s' % filename)

3. Corriendo el spider

In [ ]:
$scrapy crawl quotes

se crearán 2 ficheros nuevos con la info de las urls a la altura de donde estemos posicionados en la carpeta 'tutorial'.
1. quotes-1.html
2. quotes-2.html

# Ejemplo 2, Almacenar datos ( json, csv, jl )

In [ ]:
# crawl es el nombre de spider
# quotes.json es el nombre del archivo que se creará y su extención
$scrapy crawl quotes -o quotes.json

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('small.author::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

# Ejemplo 3, Siguiendo enlaces.

Lo primero es extraer el enlace a la página que queremos seguir. Al examinar nuestra página, podemos ver que hay un enlace a la página siguiente ( línea 18 a 21 )

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('small.author::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

        next_page = response.css('li.next a::attr(href)').extract_first()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

In [ ]:
$scrapy crawl quotes -o Siguiendo_enlaces.csv

# Ejemplo 4, Extraer información del autor.

Esta araña comenzará desde la página principal, seguirá todos los enlaces a las páginas de los autores, en la pagina principal de cada autor extraerá: name, birthdate y bio.

In [ ]:
import scrapy


class AuthorSpider(scrapy.Spider):
    name = 'author'

    start_urls = ['http://quotes.toscrape.com/']

    def parse(self, response):
        # 1) seguir enlaces a páginas de autor
        for href in response.css('.author + a::attr(href)'):
            yield response.follow(href, self.parse_author)

        # 2) seguir los enlaces de paginación
        for href in response.css('li.next a::attr(href)'):
            yield response.follow(href, self.parse) 

    def parse_author(self, response):
        # Función auxiliar para extraer y limpiar los datos de una consulta CSS 
        # y genera el dict con los datos del autor.
        def extract_with_css(query):
            return response.css(query).extract_first().strip()

        yield {
            'name': extract_with_css('h3.author-title::text'),
            'birthdate': extract_with_css('.author-born-date::text'),
            'bio': extract_with_css('.author-description::text'),
        }
        

In [ ]:
$scrapy crawl author -o Siguiendo_enlaces.csv

# Ejemplo 5, Argumentos de línea de comando

Se puede proporcionar argumentos de línea de comando en la araña usando la opción -a cuando las ejecuta:  
en ete caso queremos acceder a la siguiente url:  
http://quotes.toscrape.com/tag/humor/

In [ ]:
$scrapy crawl quotes -o argumentos_en_linea_comando.json -a tag=humor

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        url = 'http://quotes.toscrape.com/'
        tag = getattr(self, 'tag', None)
        if tag is not None:
            url = url + 'tag/' + tag
        yield scrapy.Request(url, self.parse)

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('small.author::text').extract_first(),
            }

        next_page = response.css('li.next a::attr(href)').extract_first()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

# Ejemplo 6, usando selectores CSS.
código disponible en: https://github.com/scrapy/quotesbot

In [ ]:
$scrapy crawl toscrape-css -o argumentos_en_linea_comando.json

In [ ]:
# -*- coding: utf-8 -*-
import scrapy


class ToScrapeCSSSpider(scrapy.Spider):
    name = "toscrape-css"
    start_urls = [
        'http://quotes.toscrape.com/',
    ]

    def parse(self, response):
        for quote in response.css("div.quote"):
            yield {
                'text': quote.css("span.text::text").extract_first(),
                'author': quote.css("small.author::text").extract_first(),
                'tags': quote.css("div.tags > a.tag::text").extract()
            }

        next_page_url = response.css("li.next > a::attr(href)").extract_first()
        if next_page_url is not None:
            yield scrapy.Request(response.urljoin(next_page_url))

# Ejemplo 7, usando selectores xpath.
código disponible en: https://github.com/scrapy/quotesbot

In [ ]:
# -*- coding: utf-8 -*-
import scrapy


class ToScrapeSpiderXPath(scrapy.Spider):
    name = 'toscrape-xpath'
    start_urls = [
        'http://quotes.toscrape.com/',
    ]

    def parse(self, response):
        for quote in response.xpath('//div[@class="quote"]'):
            yield {
                'text': quote.xpath('./span[@class="text"]/text()').extract_first(),
                'author': quote.xpath('.//small[@class="author"]/text()').extract_first(),
                'tags': quote.xpath('.//div[@class="tags"]/a[@class="tag"]/text()').extract()
            }

        next_page_url = response.xpath('//li[@class="next"]/a/@href').extract_first()
        if next_page_url is not None:
            yield scrapy.Request(response.urljoin(next_page_url))



# Ejemplo 8, Raspando trabajando.cl
### https://www.trabajando.cl

In [ ]:
# -*- coding: utf-8 -*-
# scrapy crawl trabajando -o trabajando.csv
import scrapy


class TrabajoSpider(scrapy.Spider):
    name = 'trabajando'

    start_urls = ['https://www.trabajando.cl/jobs/home/']

    def parse(self, response):
        # 1) seguir enlaces a páginas de postulacion
        for href in response.xpath('//*[@id="ofertas_slider"]/ul/li/div/div/div/div/h2/a/@href'):
            yield response.follow(href, self.parse_author)

        # 2) seguir los enlaces de pagina siguiente
        for href in response.xpath('//*[@id="nextPageEmpresa"]/@href'):
            yield response.follow(href, self.parse)

    def parse_author(self, response):
        # Función auxiliar para extraer y limpiar los datos de una consulta CSS
        # y genera el dict con los datos del autor.
        def extract_with_xpath(query):
            return response.xpath(query).extract_first().strip()

        yield {
            #'Link': extract_with_xpath(href),

            'Descripcion': extract_with_xpath('//*[@id="detalle_oferta"]/div/div[4]/p/text()'),
            'Funciones': extract_with_xpath('//*[@id="detalle_oferta"]/div/div[4]/p/text()'),
            'Area de desempeno': extract_with_xpath('//*[@id="detalle_oferta"]/div/div[7]/div[1]/div[2]/text()'),
            'Region': extract_with_xpath('//*[@id="detalle_oferta"]/div/div[7]/div[2]/div[2]/text()'),
            'Lugar': extract_with_xpath('//*[@id="detalle_oferta"]/div/div[7]/div[4]/div[2]/text()'),
            'Publicado': extract_with_xpath('//*[@id="detalle_oferta"]/div/div[3]/div/div[3]/h4/text()'),
            'Empresa': extract_with_xpath('//*[@id="detalle_oferta"]/div/div[2]/div[4]/h4/a/text()'),
            'Titulo': extract_with_xpath('//*[@id="detalle_oferta"]/div/div[2]/div[2]/h1/text()'),
        }
